## Notebook 3 - All locations
### **Predicting Dengue Fever Incidence and Disease Dynamics under Climate Change in Southeast Asia**
### Master's thesis by Josephine Lutter, supervised by Professor Roberto Henriques

## Table of Contents
<ul>
  <li><a href="#1.-Import">1. Import</a></li>
  <li><a href="#2.-Data-Exploration">2. Data Exploration</a>
    <ul>
      <li><a href="#a.-df_3">a. df_3</a></li>
    </ul>
  </li>
  <li><a href="#3.-Data-Preparation">3. Data Preparation</a>
    <ul>
      <li><a href="#a.-Feature-Removal">a. Feature Removal</a></li>
      <li><a href="#b.-Outlier-Detection">b. Outlier Detection</a></li>
      <li><a href="#c.-Outlier-Removal">c. Outlier Removal</a></li>
      <li><a href="#d.-Feature-Creation">d. Feature Creation</a></li>
      <li><a href="#e.-Data-Encoding">e. Data Encoding</a></li>
      <li><a href="#f.-Data-Partition">f. Data Partition</a></li>
      <li><a href="#g.-Data-Normalization">g. Data Normalization</a></li>
    </ul>
  </li>
  <li><a href="#4.-Feature-Selection">4. Feature Selection</a></li>
  <li><a href="#5.-Predictive-Modeling">5. Predictive Modeling</a>
    <ul>
      <li><a href="#5.1-Hyperparameter-Tuning">5.1 Hyperparameter Tuning</a>
        <ul>
          <li><a href="#a.-Scale-invariant-models">a. Scale-invariant models</a></li>
          <li><a href="#b.-Scale-variant-models">b. Scale-variant models</a></li>
        </ul>
      </li>
      <li><a href="#5.2-Cross-Validation-and-Model-Evaluation-on-Training-Set">5.2 Cross-Validation and Model Evaluation on Training Set</a>
        <ul>
          <li><a href="#a.-Scale-invariant-models">a. Scale-invariant models</a></li>
          <li><a href="#b.-Scale-variant-models">b. Scale-variant models</a></li>
        </ul>
      </li>
      <li><a href="#5.3-Final-Training-and-Prediction-on-Test-Set">5.3 Final Training and Prediction on Test Set</a>
        <ul>
          <li><a href="#a.-Scale-invariant-models">a. Scale-invariant models</a></li>
          <li><a href="#b.-Scale-variant-models">b. Scale-variant models</a></li>
        </ul>
      </li>
    </ul>
  </li>
</ul>

## 1. Import

In [ ]:
# Imports for Data Exploration
import os
import math
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Imports for Data Preprocessing and Engineering
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.feature_selection import RFE, SequentialFeatureSelector, SelectFromModel
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LassoCV
from xgboost import XGBRegressor

# Imports for Modeling
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint, uniform
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

In [ ]:
print(xgb.__version__)
print(sklearn.__version__)

In [ ]:
# Integration of Excel files that have been pre-structured using Mircosoft Excel to guarantee consistent structure

file = "/Users/Fine/Documents/Master Business Analytics/Thesis/Research Data/Final Data/Combined/All locations combined.xlsx"

# This notebook will process the combined and aggregated variables across all locations
# Two datasets can be analyzed, with and without location-specific outlier capping
#df_3 = pd.read_excel(file, sheet_name='Without outlier capping')
df_3 = pd.read_excel(file, sheet_name='With outlier capping')

# Define the location for plots
location = 'Southeast Asia'

# Set the Date variable as index, round and sort
df_3.set_index("Date", inplace=True)
df_3 = df_3.round(4).sort_index()

## 2. Data Exploration
This notebook will address the combined and aggregated environmental variables used for the modeling. Initially, the data is deeply explored to check for inconsistencies and analyzed using statistical techniques to further investigate the distribution and structure.

#### The combined and aggregated environmental variables for the modeling

| Variable           | Unit                 | Description                                                              |
|--------------------|----------------------|--------------------------------------------------------------------------|
| Min_Daily_Prcp     | Millimeters (mm)     | Minimum daily precipitation                                               |
| Max_Daily_Prcp     | Millimeters (mm)     | Maximum daily precipitation                                               |
| Monthly_Avg_Prcp   | Millimeters (mm)     | Monthly average precipitation based on daily temperature reporting       |
| Monthly_Total_Prcp | Millimeters (mm)     | Monthly total precipitation based on cumulated daily precipitation        |
| Monthly_Avg_Temp   | Degrees Celsius (°C) | Monthly average temperature based on daily temperature reporting          |
| Min_Daily_Temp     | Degrees Celsius (°C) | Minimum daily temperature of the respective month                         |
| Max_Daily_Temp     | Degrees Celsius (°C) | Maximum daily temperature of the respective month                         |
| Min_Average_Temp   | Degrees Celsius (°C) | Minimum value of the average daily temperature                            |
| Max_Average_Temp   | Degrees Celsius (°C) | Maximum value of the average daily temperature                            |
| N_Raining_Days     | Degrees Celsius (°C) | Cumulative number of raining days of the respective month                 |

### a. df_3

#### Exploratory functions

In [ ]:
# Exploring the data types and structure using exploratory functions
df_3.head()

In [ ]:
# Exploring the data types and structure using exploratory functions
df_3.info()

In [ ]:
# Exploring the data types and structure using exploratory functions
df_3.describe().T

In [ ]:
# Besides data visualization, there are multiple tests to verify if a time series is stationary
# In the following, the Augmented Dickey-Fuller (ADF) test is performed
# Inspiration: https://medium.com/@JDEconomics/how-to-test-for-stationarity-in-time-series-data-using-python-44d82890aa9d
# Understanding trends and seasonality is fundamental as they provide insights into long-term patterns and recurring fluctuations
# Further techniques such as decomposition, differencing, and detrending can be utilized, facilitating the analysis of stationary data

# Specifying the subset of variables without "Min_Daily_Prcp" due to being constant
environmental_variables = [
    "Max_Daily_Prcp", "Monthly_Avg_Prcp", "Monthly_Total_Prcp",
    "Monthly_Avg_Temp", "Min_Daily_Temp", "Max_Daily_Temp",
    "Min_Average_Temp", "Max_Average_Temp", "N_Raining_Days"
]

# List to store non-stationary variables
non_stationary = []

# Perform ADF test for selected variables
for variable in environmental_variables:
    result = adfuller(df_3[variable])
    print(f'{variable} shows a p-value of: {result[1]:0.3f}') # Rounded here, therefore shows 0.000 in some instances
    if result[1] <= 0.05:
        print('Test Result: \033[92mStationary\033[0m') 
    else:
        print('Test Result: \033[91mNon-Stationary\033[0m')
        non_stationary.append(variable)

#### Result

All variables were determined to be stationary; consequently, statistical characteristics did not change over time. No further methods are required to force stationarity, which is good because changing patterns and external forces such as climate change are relevant to the underlying project. Non-stationarity could harm model performance. 

#### Exploratory visualizations

In [ ]:
# Plot cumulative target variable Incidence
# Interesting but considered careful interpretation because years vary in data availability
# With 2016 including data points from six locations, 2023 from one, and 2017-2020 being supported by 17 locations

# Group by Month and Year, then sum the Incidence for each group
cumulative_incidence = df_3.groupby(['Year', 'Month'])['Incidence'].sum().reset_index()

# Increase figure size
plt.figure(figsize=(12, 8))

# Plot
plt.plot(cumulative_incidence['Year'], cumulative_incidence['Incidence'], marker='o', linestyle='-')
plt.title('Cumulative Incidence Over Time', fontsize=18)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Cumulative Incidence', fontsize=14)
plt.xticks()
plt.yticks()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# To drive interpretability, the mean incidence is displayed

# Calculate yearly mean incidence
yearly_mean_incidence = df_3.groupby('Year')['Incidence'].mean().reset_index()

# Increase figure size
plt.figure(figsize=(12, 8))

# Plot
plt.plot(yearly_mean_incidence['Year'], yearly_mean_incidence['Incidence'], marker='o', linestyle='-')
plt.title('Yearly Mean Incidence', fontsize=18)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Incidence', fontsize=14)
plt.xticks()
plt.yticks()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Histogram for univariate analysis
# Draw a histogram for each numerical variable to explore the general data distribution, structure, and detect outliers of df_3
# Subsequentally, data will be scaled. Therefore, outlier treatment is significant 

# Append the new environmental variable Min_Daily_Temp for the overall exploratory analysis
environmental_variables.append("Min_Daily_Prcp")

sns.set(style="whitegrid")

fig, axes = plt.subplots(4, math.ceil(len(environmental_variables) / 4), figsize=(20, 25))

for ax, feat in zip(axes.flatten(), environmental_variables):
    ax.set_ylabel('Count')
    # Plot histogram
    counts, bins, patches = ax.hist(df_3[feat].dropna(), alpha=0.6) 
    
    # Adding KDE and statistics
    sns.histplot(data=df_3, x=feat, bins=bins, kde=True, ax=ax, line_kws={'lw': 3})
    
    # Adding annotations for counts
    for count, patch in zip(counts, patches):
        ax.annotate(str(int(count)), xy=(patch.get_x() + patch.get_width() / 2, patch.get_height()),
                    xytext=(0, 5), textcoords='offset points',
                    ha='center', fontsize=12, fontweight='bold')
    
    # Adding mean and standard deviation text
    ax.text(0.725, 0.9, f'$\mu={df_3[feat].mean():0.2f}$\n$\sigma={df_3[feat].std():0.2f}$',
            transform=ax.transAxes, fontsize=15, verticalalignment='top', color='white',
            bbox=dict(boxstyle='round'))
    
plt.suptitle(f"{location}: Monthly Environmental Variables' Histograms", fontsize=18)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.4, wspace=0.4)
plt.savefig(os.path.join(f"{location}_monthly_environmental_variables_histograms.png"), dpi=200)
plt.show()

In [ ]:
# Boxplots for univaraite analysis
# Draw a histogram for each numerical column to explore the general distribution, structure, and detect outliers of df_3
# Subsequentally, data will be scaled. Therefore, outlier treatment is significant 
# Inspiration: https://medium.com/analytics-vidhya/how-to-remove-outliers-for-machine-learning-24620c4657e8

fig, axes = plt.subplots(4, math.ceil(len(environmental_variables) / 4), figsize=(20, 25))

for ax, feat in zip(axes.flatten(), environmental_variables):
    sns.boxplot(x=df_3[feat], ax=ax, color="b")
    
    # Adding mean and standard deviation text
    ax.text(0.75, 0.9, f'$\mu={df_3[feat].mean():0.2f}$\n$\sigma={df_3[feat].std():0.2f}$',
            transform=ax.transAxes, fontsize=15, verticalalignment='top', color='white',
            bbox=dict(boxstyle='round', facecolor='b'))
    
plt.suptitle(f"{location}: Monthly Environmental Variables' Box Plots", fontsize=18)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.4, wspace=0.4)
plt.savefig(os.path.join(f"{location}_monthly_environmental_variables_boxplots.png"), dpi=200)
plt.show()

In [ ]:
#Check rows of the strongest outliers of Monthly_Total_Prcp, that seem extrem
df_3.sort_values(by='Monthly_Total_Prcp', ascending=False).head(13)

In [ ]:
# Line plot of the df_3 combined variables

fig, axes = plt.subplots(4, math.ceil(len(environmental_variables) / 4), figsize=(20, 23))

for ax, feat in zip(axes.flatten(), environmental_variables):
    ax.set_ylabel('Value')
    # Plot line plot
    ax.plot(df_3[feat], color="g", alpha=0.6) 
    
    # Adding mean and standard deviation text
    ax.text(0.725, 0.9, f'$\mu={df_3[feat].mean():0.2f}$\n$\sigma={df_3[feat].std():0.2f}$',
            transform=ax.transAxes, fontsize=15, verticalalignment='top', color='white',
            bbox=dict(boxstyle='round', facecolor='g'))
    
    ax.set_title(feat, fontsize=12)
    
plt.suptitle(f"{location}: Monthly Environmental Variables' Line Plots", fontsize=18)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.4, wspace=0.4)
plt.savefig(os.path.join(f"{location}_monthly_environmental_variables_lineplots.png"), dpi=200)
plt.show()

In [ ]:
# Seasonality and trend exploration - not applied here as variables are stationary
# Perform seasonal decomposition using the list of non-stationary variables

# Perform seasonal decomposition and plot for each non-stationary variable
for variable in non_stationary:
    result = seasonal_decompose(df_3[variable], model='multiplicative')
    
    # Access the components of the decomposition
    trend = result.trend
    seasonal = result.seasonal
    residual = result.resid
    
    # Plot the components for the current variable
    plt.figure(figsize=(12, 8))
    plt.suptitle(f"Seasonal Decomposition - {variable}", fontsize=16)  # Heading for all four plots
    
    plt.subplot(4, 1, 1)
    plt.plot(df_3.index, df_3[variable], label='Original', color='orange')
    plt.legend(loc='upper left')

    plt.subplot(4, 1, 2)
    plt.plot(df_3.index, trend, label='Trend', color='blue')
    plt.legend(loc='upper left')

    plt.subplot(4, 1, 3)
    plt.plot(df_3.index, seasonal, label='Seasonal', color='red')
    plt.legend(loc='upper left')

    plt.subplot(4, 1, 4)
    plt.plot(df_3.index, residual, label='Residual', color='green')
    plt.legend(loc='upper left')

    plt.tight_layout()
    plt.savefig(os.path.join(f"{location}_seasonal_decomposition.png"), dpi=200)
    plt.show()

In [ ]:
# Bivariate Analysis with the target variable

# Iterate the environmental variables
for variable in environmental_variables:
    plt.figure(figsize=(15, 8))
    
    # Plot environmental variable against target variable
    sns.lineplot(data=df_3, x='Date', y=variable, color='green', label=variable)
    
    # Plot target variable Incidence
    ax2 = plt.twinx()
    sns.lineplot(data=df_3, x='Date', y='Incidence', color='orange', ax=ax2, label='Incidence')

    plt.title(f"{location}: Bivariate Analysis of Monthly Incidence and {variable}", fontsize=18)
    plt.xlabel('Date')
    plt.legend(loc='upper left')
    plt.grid(False)
    plt.savefig(os.path.join(f"{location}_bivariate_analysis.png"), dpi=200)
    plt.show()

In [ ]:
# Checking for seasonal, monthly dependencies using a visualization technique

# Grouping the data by year for extraction
grouped_data = df_3.groupby('Year')['Monthly_Total_Prcp'].apply(list)

plt.figure(figsize=(12, 6))

for year, values in grouped_data.items():
    months = df_3[df_3['Year'] == year]['Month']
    plt.plot(months, values, label=str(year))

plt.title(f"{location}: Observing Monthly Dependencies using Yearly Data", fontsize=18)
plt.xlabel('Month of the Year')
plt.ylabel('Value')
plt.legend(title='Year')
plt.grid(True)
plt.savefig(os.path.join(f"{location}_seasonal_dependencies_weekly_incidence.png"), dpi=200)
plt.show()

In [ ]:
# Visualize the correlation using a correlation one-sided matrix

# Select the variables to compute the correlation
correlation_variables = ["Min_Daily_Prcp", "Max_Daily_Prcp", "Monthly_Avg_Prcp",
                         "Monthly_Total_Prcp", "Monthly_Avg_Temp", "Min_Daily_Temp", 
                         "Max_Daily_Temp", "Min_Average_Temp", "Max_Average_Temp", 
                         "N_Raining_Days", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt="0.2f", linewidths=0.5)
plt.title(f"Correlation Matrix of Incidence Rate and Monthly Environmental Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_monthly_1.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

#### Result

The feature Min_Daily_Prcp, which represents the minimum daily precipitation, shows limited significance with 0 mm in most cases. In addition, the variables Monthly_Average_Prcp, Min_Average_Temp, and Max_Average_Temp might lead to multicollinearity due to high correlation, which adds redundancy. Overall, the correlation matrix shows low significance with the target variable.

The univariate analysis shows remaining outliers for the dataset with location-specific outlier capping. Consequently, aside from data aggregation and the capping of relative outliers, the model performance might benefit from addressing extremes on a multi-location level. This will be tested at a later stage. 

Considering the total dengue infection cycle lasts 4-7 weeks, the bivariate analysis does not reveal a strong relationship with the target variable. An association of seasonality can not be derived. 

## 3. Data Preparation

In the first data preprocessing, missing values of the raw data have been imputed. In notebook 2, outliers were capped in location-specific datasets. This step performs further preprocessing, such as feature deletion, outlier detection and removal on a multi-location level, encoding, partition, and scaling.  Additionally, feature engineering is performed by creating lags of the environmental variables.

Having identified some extreme outliers in the EDA, robust scaling will be interesting because it is robust to outliers. However, the MinMax scaler will also be applied to compare modeling outcomes as this thesis focuses on applying a multitude of methods.

In [ ]:
# Printing environmental variables
environmental_variables

### a. Feature Removal

Removing irrelevant features that are identified irrlevant.

In [ ]:
# Removing Min_Daily_Prcp because of insufficient interpretability
# Removing Monthly_Avg_Prcp, Min_Average_Temp, Max_Average_Temp due to high correlation

df_3 = df_3.drop(columns=['Min_Daily_Prcp', 'Monthly_Avg_Prcp', 'Min_Average_Temp', 'Max_Average_Temp'])

# Simultaneous removal of the list
for var in ['Min_Daily_Prcp', 'Monthly_Avg_Prcp', 'Min_Average_Temp', 'Max_Average_Temp']:
    if var in environmental_variables:
        environmental_variables.remove(var)

# Print updated list
environmental_variables

In [ ]:
# Visualize the correlation using a correlation one-sided matrix with the updated variables

# Select the variables to compute the correlation
correlation_variables = ["Max_Daily_Prcp", "Monthly_Total_Prcp", "Min_Daily_Temp", 
                         "Max_Daily_Temp", "Monthly_Avg_Temp", 
                         "N_Raining_Days", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt="0.2f", linewidths=0.5)
plt.title(f"Correlation Matrix of Incidence Rate and Monthly Environmental Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_monthly_2.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

### b. Outlier Detection 

The main objective is to find a balance by reducing noise and allowing the dynamic nature of the environmental features. As emphasized, extreme weather events such as flushing tremendously affect mosquito habitat, lifecycle, and disease transmission. Accordingly, methods like outlier removal and trimming through IQR were disregarded.

At first, the standard deviation method is conducted to explore outliers with a standard deviation far from the mean. Additionally, the z-score method is utilized for detection, standardizing the data, and identifying outliers based on their distance from the mean in terms of standard deviations. Outlier capping would not be beneficial for the identification as it determines outliers for all variables within a specified percentile. Therefore, it simply defines the most profound values. Nevertheless, outlier capping is an elegant approach for outlier treatment, preserving integrity, improving interpretability, and avoiding overfitting. There are two approaches: Floor capping values below a certain threshold and ceiling replacing values above the threshold.

- Source Capping: https://medium.com/analytics-vidhya/outlier-detection-in-machine-learning-382557c775aa
- Source IQR: https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/

In [ ]:
# Using the standard deviation method to detect outliers

# Calculate mean and standard deviation only for environmental variables
means = df_3[environmental_variables].mean()
stds = df_3[environmental_variables].std()

# Defining the threshold
threshold = 4

# Identify outliers for each environmental variable
outliers = {}
for variable in environmental_variables:
    outlier_indices = np.abs(df_3[variable] - means[variable]) > threshold * stds[variable]
    outliers[variable] = df_3.loc[outlier_indices, variable].tolist()

print("Outliers identified using the Standard Deviation Method:")
for variable, values in outliers.items():
    values.sort()
    print(f"{variable}: {values}")

In [ ]:
# Define a threshold for z-score beyond which a data point is considered an outlier

# Calculate z-scores for each environmental variable
z_scores = (df_3[environmental_variables] - df_3[environmental_variables].mean()) / df_3[environmental_variables].std()

# Defining the threshold
threshold = 4

# Identify outliers for each environmental variable
outliers = {}
for variable in environmental_variables:
    outlier_indices = np.abs(z_scores[variable]) > threshold
    outliers[variable] = df_3.loc[outlier_indices, variable].tolist()

print("Outliers identified using the Z-Score Method:")
for variable, values in outliers.items():
    values.sort()
    print(f"{variable}: {values}")

In [ ]:
# Check quantiles before applying outlier capping
# Quartiles divide the data into four equal parts with Q2 being the median

# Dictionary to store outliers for each variable
outliers = {}

# Iterate over each variable
for variable in environmental_variables:
    
    # Calculate the first and third quartile of each variable
    Q1 = df_3[variable].quantile(0.10)
    Q3 = df_3[variable].quantile(0.90)
            
    # Calculating the IQR range
    IQR = Q3 - Q1

    # Calculating the lower and upper bound/whisker
    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)

    # Identify outliers
    outlier_indices = (df_3[variable] < lower_bound) | (df_3[variable] > upper_bound)
    outliers[variable] = df_3.loc[outlier_indices, variable].tolist()

# Print outliers for each variable outside the loop
print("Outliers identified using the IQR Method:")
for variable, values in outliers.items():
    values.sort()
    print(f"{variable}: {values}")

### c. Outlier Removal

Depending on the use case, the following function needs to be commented / not commented.

In [ ]:
# Capping Functions
# https://wellsr.com/python/outlier-data-handling-with-python/#:~:text=Capping%20Outliers%20using%20Fixed%20Quantiles&text=In%20such%20cases%20you%20can,the%20records%20in%20the%20dataset.

def outlier_capping(data, outlier_variables):
    # Create a copy of the original data to avoid modifying it directly
    # data = data.copy()

    # Loops through each feature with outliers.
    for variable in outlier_variables:

            # Defines the first and third quantile of each feature.
            Q1 = data[variable].quantile(0.15)
            Q3 = data[variable].quantile(0.85)

            # Calculating the IQR range.
            IQR = Q3 - Q1

            # Calculating the lower and upper bound/whisker
            lower_bound = Q1 - (1.5 * IQR)
            upper_bound = Q3 + (1.5 * IQR)

            # Replaces all values below the lower bound and
            # above the upper bound with the lower or upper bound values.
            data[variable] = np.where(data[variable] > upper_bound, upper_bound,
                                        np.where(data[variable] < lower_bound, lower_bound,
                                                  data[variable]))
    return data

In [ ]:
#outlier_capping(df_3, environmental_variables)

In [ ]:
# Histogram to control outlier removal

fig, axes = plt.subplots(3, math.ceil(len(environmental_variables) / 4), figsize=(20, 23))

for ax, feat in zip(axes.flatten(), environmental_variables):
    ax.set_ylabel('Count')
    # Plot histogram
    counts, bins, patches = ax.hist(df_3[feat].dropna(), alpha=0.6) 
    
    # Adding KDE and statistics
    sns.histplot(data=df_3, x=feat, bins=bins, kde=True, ax=ax, line_kws={'lw': 3})
    
    # Adding annotations for counts
    for count, patch in zip(counts, patches):
        ax.annotate(str(int(count)), xy=(patch.get_x() + patch.get_width() / 2, patch.get_height()),
                    xytext=(0, 5), textcoords='offset points',
                    ha='center', fontsize=12, fontweight='bold')
    
    # Adding mean and standard deviation text
    ax.text(0.725, 0.9, f'$\mu={df_3[feat].mean():.2f}$\n$\sigma={df_3[feat].std():.2f}$',
            transform=ax.transAxes, fontsize=15, verticalalignment='top', color='white',
            bbox=dict(boxstyle='round'))
    
plt.suptitle(f"Monthly Environmental Variables' Histograms after Outlier Capping", fontsize=18)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.4, wspace=0.4)
plt.savefig(os.path.join(f"{location}_monthly_environmental_variables_histograms_outlier_removal.png"), dpi=200)
plt.show()

#### Results

Outlier detection was performed for each location using the z-score and standard deviation method. Observations beyond a threshold of 2 or 3 standard deviations from the mean are considered outliers. Here, I experimented with a threshold of 3 and 4 and concluded that 4 was a better fit. The outcome of the outlier detection was compared with the IQR method. Visualizations were used to support the assessment. In conclusion, the 15/85 split was chosen to maintain analytical integrity while tolerating the complexities of environmental data.

### d. Feature Creation

Having defined relevant environmental variables in the previous sections, the objective here is to create lagged variables for optimal time-series forecasting. At first, the lagged intervals will be determined. As discussed in the literature review, the total dengue infection cycle lasts 4-7 weeks. Moreover, as elaborated, environmental factors have an immense influence on the mosquito habitat and lifecycle. Consequently, lagged variables will be established for each environmental factor at multiple intervals prior to the current observation, considering, that the data is aggregated on a monthly scale.

In [ ]:
# Creation of lagged features through transformation
# Lag intervals from 1 to 6 months
# Meaning missing values will be created in the first five instances as the data is not given
# Rows with missing values will be excluded to avoid subsequent issues

# Define interval
lagged_intervals = range(1, 7)
new_features = []

# Create lags
for variable in environmental_variables:
    for lag in lagged_intervals:
        new_feature_name = f"{variable}_lag_{lag}"
        df_3[new_feature_name] = df_3[variable].shift(lag)
        new_features.append(new_feature_name)

# Update environmental_variables
environmental_variables.extend(new_features)

# Drop rows with missing values as they are represented in the lags
df_3.dropna(inplace=True)

# Visualize the engineered data 
df_3.head()

In [ ]:
# Visualize the correlation of lagged variables using a correlation one-sided matrix

# Select the variables to compute the correlation
correlation_variables = ["Max_Daily_Prcp", "Max_Daily_Prcp_lag_1", "Max_Daily_Prcp_lag_2",
                         "Max_Daily_Prcp_lag_3", "Max_Daily_Prcp_lag_4", "Max_Daily_Prcp_lag_5",
                         "Max_Daily_Prcp_lag_6", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt="0.2f", linewidths=0.5)
plt.title(f"Correlation Matrix of Incidence Rate and Max_Daily_Prcp Lagged Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_lagged.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the correlation of lagged variables using a correlation one-sided matrix

# Select the variables to compute the correlation
correlation_variables = ["Monthly_Total_Prcp", "Monthly_Total_Prcp_lag_1", "Monthly_Total_Prcp_lag_2",
                         "Monthly_Total_Prcp_lag_3", "Monthly_Total_Prcp_lag_4", "Monthly_Total_Prcp_lag_5",
                         "Monthly_Total_Prcp_lag_6", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.5)
plt.title(f"Correlation Matrix of Incidence Rate and Monthly_Total_Prcp Lagged Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_lagged.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the correlation of lagged variables using a correlation one-sided matrix

# Select the variables to compute the correlation
correlation_variables = ["Min_Daily_Temp", "Min_Daily_Temp_lag_1", "Min_Daily_Temp_lag_2",
                         "Min_Daily_Temp_lag_3", "Min_Daily_Temp_lag_4", "Min_Daily_Temp_lag_5",
                         "Min_Daily_Temp_lag_6", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.5)
plt.title(f"Correlation Matrix of Incidence Rate and Min_Daily_Temp Lagged Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_lagged.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the correlation of lagged variables using a correlation one-sided matrix

# Select the variables to compute the correlation
correlation_variables = ["Max_Daily_Temp", "Max_Daily_Temp_lag_1", "Max_Daily_Temp_lag_2",
                         "Max_Daily_Temp_lag_3", "Max_Daily_Temp_lag_4", "Max_Daily_Temp_lag_5",
                         "Max_Daily_Temp_lag_6", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.5)
plt.title(f"Correlation Matrix of Incidence Rate and Max_Daily_Temp Lagged Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_lagged.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the correlation of lagged variables using a correlation one-sided matrix

# Select the variables to compute the correlation
correlation_variables = ["Monthly_Avg_Temp", "Monthly_Avg_Temp_lag_1", "Monthly_Avg_Temp_lag_2",
                         "Monthly_Avg_Temp_lag_3", "Monthly_Avg_Temp_lag_4", "Monthly_Avg_Temp_lag_5",
                         "Monthly_Avg_Temp_lag_6", "Incidence Rate"]

# Create a correlation matrix
correlation_matrix = df_3[correlation_variables].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Heatmap creation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.5)
plt.title(f"Correlation Matrix of Incidence Rate and Monthly_Avg_Temp Lagged Variables", fontsize=18)
plt.savefig(os.path.join(f"{location}_correlation_matrix_lagged.png"), dpi=200, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# Check how many features are established after feature engineering
len(environmental_variables)

In [ ]:
# Print list with lagged variables
environmental_variables

In [ ]:
# Heatmap of the correlation of incidence with the environmental variables and their lagged observations

# Calculate the correlation matrix for the selected variables
correlation_matrix = df_3[environmental_variables + ['Incidence Rate']].corr()

# Filter the correlation matrix to include only the correlations with the target variable
filtered_corr_matrix = correlation_matrix.loc[environmental_variables, 'Incidence Rate']

# Sort the correlation values in descending order
sorted_corr_matrix = filtered_corr_matrix.sort_values(ascending=False)

# Plotting the heatmap
plt.figure(figsize=(12, 18))
ax = sns.heatmap(sorted_corr_matrix.to_frame(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, linecolor='black')
ax.set_title('Correlation of Incidence Rate with Monthly Environmental Variables and Lagged Observations', fontsize=18)
ax.set_xlabel('')
ax.set_ylabel('Environmental Variables', fontsize=14)
ax.set_xticklabels(['Incidence Rate'], fontsize=12)

# Save the figure
plt.savefig(os.path.join(f"{location}_correlation_heatmap_all_variables.png"), dpi=300, bbox_inches='tight')
plt.show()

### e. Data Encoding
Encoding of categorical variable "Name". Source of inspiration: https://medium.com/aiskunks/categorical-data-encoding-techniques-d6296697a40f

In [ ]:
# Fit and transform the variable Name with LabelEncoder
df_3['Location Code'] = LabelEncoder().fit_transform(df_3['Name'])

In [ ]:
# Print location names and their respective location codes
df_3[['Name', 'Location Code']].drop_duplicates().sort_values('Location Code')

### f. Data Partition

Before normalizing data by applying different scaling methods, data is split to compare scaled and unscaled data in the modeling section directly. The aim is to carefully split the time-series data with respect to the index Date to avoid data leakage. Different approaches will be tested, with one holistic strategy and a location-specific individual approach. The later applied time-series cross-validation aligns with an expanding window training approach.

Sources of inspiration:
- https://medium.com/@content_33167/sampling-techniques-for-time-series-data-analysis-8d76e423baed
- https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8
- https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4
- https://medium.com/@Stan_DS/timeseries-split-with-sklearn-tips-8162c83612b9
- https://forecastegy.com/posts/time-series-cross-validation-python/
- https://tomerkatzav.medium.com/split-time-series-dataset-826b7dc39cd9
- https://towardsdatascience.com/cross-validation-explained-evaluating-estimator-performance-e51e5430ff85
- https://medium.com/eatpredlove/time-series-cross-validation-a-walk-forward-approach-in-python-8534dd1db51a
- https://gghantiwala.medium.com/can-using-a-different-way-to-split-your-time-series-data-improve-model-performance-3f5042f13dff

#### Holistic approach

Presented here is the aligned time-series of the holistic model. The initially implemented time-based split for each location raised the concern of data leakage due to non-aligned time ranges within the training process.

In [ ]:
# Holistic, dependent approach with cut-off

# Create lists
train_dfs, val_dfs, test_dfs, y_train_dfs, y_val_dfs, y_test_dfs = [], [], [], [], [], []

# Define the cutoff dates
validation_cutoff = pd.Timestamp('2019-01-01')
test_cutoff = pd.Timestamp('2020-01-01')

# Perform the split for each unique location
for location in df_3['Location Code'].unique():
    location_df = df_3[df_3['Location Code'] == location]
    features = ['Location Code'] + environmental_variables  # Assume environmental_variables is defined
    target = 'Incidence Rate'
    
    # Select features and target for the current location
    X_location = location_df[features]
    y_location = location_df[target]

    # Split data into train, validation, and test sets based on the cutoff
    X_train = X_location[(X_location.index < validation_cutoff)]
    X_val = X_location[(X_location.index >= validation_cutoff) & (X_location.index < test_cutoff)]
    X_test = X_location[X_location.index >= test_cutoff]

    y_train = y_location[(y_location.index < validation_cutoff)]
    y_val = y_location[(y_location.index >= validation_cutoff) & (y_location.index < test_cutoff)]
    y_test = y_location[y_location.index >= test_cutoff]

    # Append the splits to their respective lists
    train_dfs.append(X_train)
    val_dfs.append(X_val)
    test_dfs.append(X_test)
    y_train_dfs.append(y_train)
    y_val_dfs.append(y_val)
    y_test_dfs.append(y_test)
    
# Concatenate the splits into training, validation, and testing sets
X_train, X_val, X_test = pd.concat(train_dfs).sort_index(), pd.concat(val_dfs).sort_index(), pd.concat(test_dfs).sort_index()
y_train, y_val, y_test = pd.concat(y_train_dfs).sort_index(), pd.concat(y_val_dfs).sort_index(), pd.concat(y_test_dfs).sort_index()

In [ ]:
# Visualization: Holistic, dependent approach with cut-off

plt.figure(figsize=(12, 8))

# Plot data for each unique location
for location in df_3['Location Code'].unique(): 
    # Plot training data
    train_subset = X_train[X_train['Location Code'] == location]
    plt.scatter(train_subset.index, [location] * len(train_subset), color='blue', alpha=0.5)
    
    # Plot validation data
    val_subset = X_val[X_val['Location Code'] == location]
    plt.scatter(val_subset.index, [location] * len(val_subset), color='green', alpha=0.5)

    # Plot testing data
    test_subset = X_test[X_test['Location Code'] == location]
    plt.scatter(test_subset.index, [location] * len(test_subset), color='red', alpha=0.5)

plt.xlabel('Date')
plt.ylabel('Location Code')
plt.title('Train-Validation-Test Split by Location Code', fontsize=18)
plt.legend(['Train', 'Validation', 'Test'])
plt.grid(True)
#plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(f"Train-Validation-Test-Split-by-Location_2.png"), dpi=200)
plt.show()

#### Individual approach

In [ ]:
# Individual, independent approach
# New: Reserving the X_train for later using lists
#Here, time-based train test split

# Prepare lists to hold split data
train_dfs, test_dfs, y_train_dfs, y_test_dfs = [], [], [], []

# Perform the split for each unique location
for location in df_3['Location Code'].unique():
    
    location_df = df_3[df_3['Location Code'] == location]
    features = ['Location Code'] + environmental_variables
    target = 'Incidence Rate'
    
    # Select features and target for the current location
    X_location = location_df[features]
    y_location = location_df[target]

    # Calculate the train size by excluding the last 12 instances, which represents a year
    train_size = len(X_location) - 12

    # Split the data into train and test sets for the current location
    X_train_val, X_test = X_location.iloc[:train_size], X_location.iloc[train_size:]
    y_train_val, y_test = y_location.iloc[:train_size], y_location.iloc[train_size:]

    # Append the splits to their respective lists
    train_dfs.append(X_train_val)
    test_dfs.append(X_test)
    y_train_dfs.append(y_train_val)
    y_test_dfs.append(y_test)
    
# Concatenate the splits into training and testing sets
X_train_val, X_test = pd.concat(train_dfs).sort_index(), pd.concat(test_dfs).sort_index()
y_train_val, y_test = pd.concat(y_train_dfs).sort_index(), pd.concat(y_test_dfs).sort_index()

In [ ]:
# Visualization: Individual, independent approach

# Visualization of the train-validation-test split by location
plt.figure(figsize=(12, 8))

# Plot data for each unique location
for location in df_3['Location Code'].unique(): 
    # Plot training data
    train_subset = X_train_val[X_train_val['Location Code'] == location]
    plt.scatter(train_subset.index, [location] * len(train_subset), color='blue', alpha=0.5)

    # Plot testing data
    test_subset = X_test[X_test['Location Code'] == location]
    plt.scatter(test_subset.index, [location] * len(test_subset), color='red', alpha=0.5)

plt.xlabel('Date')
plt.ylabel('Location Code')
plt.title('Train-Test Split by Location Code', fontsize=18)
plt.legend(['Train', 'Test'])
plt.grid(True)
plt.xticks()
plt.tight_layout()
plt.savefig(os.path.join(f"{location}:Train-Test-Split-for-each-location.png"), dpi=200)
plt.show()

### g. Data Normalization

Having identified some extreme outliers in the EDA, RobustScaler is applied because it is robust to outliers. However, the MinMaxScaler will  be applied to compare modeling outcomes as this thesis focuses on applying a multitude of methods.

In [ ]:
# Function for feature scaling

def feature_scaling(X_train, X_test, numerical_features, scaler):

    # Apply scaling
    X_train_scaled = scaler.fit_transform(X_train[numerical_features])
    X_test_scaled = scaler.transform(X_test[numerical_features])
    
    # Convert scaled arrays back to DataFrame, ensuring correct column names and index
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=numerical_features, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=numerical_features, index=X_test.index)
    
    # Combine scaled and non-scaled features
    X_train_final = pd.concat([X_train.drop(columns=numerical_features), X_train_scaled], axis=1)
    X_test_final = pd.concat([X_test.drop(columns=numerical_features), X_test_scaled], axis=1)
    
    return X_train_final, X_test_final

In [ ]:
# Function for inverse scaling

def inverse_feature_scaling(X_train_scaled, X_test_scaled, numerical_features, scaler):

    # Inverse scaling
    X_train_inverse_scaled = scaler.inverse_transform(X_train_scaled[numerical_features])
    X_test_inverse_scaled = scaler.inverse_transform(X_test_scaled[numerical_features])
    
    # Convert inverse scaled arrays back to DataFrame, ensuring correct column names and index
    X_train_inverse_scaled = pd.DataFrame(X_train_inverse_scaled, columns=numerical_features, index=X_train_scaled.index)
    X_test_inverse_scaled = pd.DataFrame(X_test_inverse_scaled, columns=numerical_features, index=X_test_scaled.index)
    
    # Combine inverse scaled and non-scaled features
    X_train_final = pd.concat([X_train_scaled.drop(columns=numerical_features), X_train_inverse_scaled], axis=1)
    X_test_final = pd.concat([X_test_scaled.drop(columns=numerical_features), X_test_inverse_scaled], axis=1)
    
    return X_train_final, X_test_final

In [ ]:
X_train_val_scaled_minmax, X_test_scaled_minmax = feature_scaling(X_train_val, X_test, environmental_variables, MinMaxScaler())

In [ ]:
X_train_val_scaled_robust, X_test_scaled_robust = feature_scaling(X_train_val, X_test, environmental_variables, RobustScaler())

## 4. Feature Selection

For the feature selection, multiple methods were explored. The main objective is to define significant environmental features that interact with disease transmission to boost model performance. In the following, the chosen methods are presented:


| Method                              | Description                                                                                           | Comment                                                                                                                             |
|-------------------------------------|-------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------|
| RFE (Recursive Feature Elimination) | The wrapper method eliminates the least significant features iteratively until the optimal subset is achieved. | Regressor and the number of features need to be prior defined, can handle multiple relationships with target variable. |
| Forward Feature Selection | This wrapper method iteratively adds the most significant features until the optimal subset is defined. | Regressor and the number of features need to be prior defined, can handle multiple relationships with target variable. |
| Random Forest | The filter method estimates feature importance based on how much the tree nodes that use the feature reduce impurity.  | Robust method for feature selection with built-in ranking, capable of handling non-linear relationships Tree-based, therefore, not sensitive to scaling. |
| XGBoost | Similar to Random Forest, XGBoost estimates feature importance based on how much the tree nodes that use the feature reduce impurity. | Also a robust method for feature selection, often used in ensemble methods. Sensitive to scaling but less than linear models. |

Comment: sklearn.feature_selection.SelectFromModel is a meta-transformer for selecting features based on importance weights.

Sources: 
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html
- https://scikit-learn.org/stable/modules/feature_selection.html
- https://towardsdatascience.com/feature-importance-and-forward-feature-selection-752638849962
- https://towardsdatascience.com/feature-selection-using-random-forest-26d7b747597f
- https://builtin.com/data-science/feature-importance
- https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/#:~:text=Forward%20feature%20selection%20involves%20iteratively,the%20most%20informative%20features%20incrementally


In [ ]:
# First method with Random Forest based on feature importance

# Initialize SelectFromModel with RandomForestRegressor
rf_selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42))

# Fit the selector on training data
rf_selector.fit(X_train_val, y_train_val)

# Get and print the selected features
rf_features = list(X_train_val.columns[(rf_selector.get_support())])
print(rf_features)

In [ ]:
# Print plot

# Access the Random Forest model to get the feature importances
feature_importances = rf_selector.estimator_.feature_importances_

# Get the indices of the features sorted by importance
sorted_indices = feature_importances.argsort()[::-1]

plt.figure(figsize=(12, 8))
plt.title('Feature Importance using RandomForestRegressor()',fontsize=18)
plt.bar(range(X_train_val.shape[1]), feature_importances[sorted_indices], align='center')
plt.xticks(range(X_train_val.shape[1]), X_train_val.columns[sorted_indices], rotation=90)
plt.ylabel('Importance')
plt.subplots_adjust(bottom=0.3)
plt.savefig(os.path.join("Feature_Importance_RF-final.png"), dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Second method, utilizing the code from above to perform similar feature selection using xgboost
# Can be tested with and without scaling

# Initialize SelectFromModel with XGBRegressor
xgb_selector = SelectFromModel(XGBRegressor(random_state=42))

# Fit the selector on training data
xgb_selector.fit(X_train_val, y_train_val)

# Get and print the selected features
xgb_features = list(X_train_val.columns[(xgb_selector.get_support())])
print(xgb_features)

In [ ]:
# Print plot

# Access the XGBoost model to get the feature importances
feature_importances = xgb_selector.estimator_.feature_importances_

# Get the indices of the features sorted by importance
sorted_indices = feature_importances.argsort()[::-1]

plt.figure(figsize=(12, 8))
plt.title('Feature Importance using XGBRegressor()', fontsize=18)
plt.bar(range(X_train_val.shape[1]), feature_importances[sorted_indices], align='center')
plt.xticks(range(X_train_val.shape[1]), X_train_val.columns[sorted_indices], rotation=90)
plt.ylabel('Importance')
plt.subplots_adjust(bottom=0.3)
plt.savefig(os.path.join("Feature_Importance_XGB-final.png"), dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Third method for feature selection, RFE
# Using data split without scaling as the random forest is scale-invariant

# Initialize the model
model = RandomForestRegressor(n_jobs=-1, random_state=42)

# Perform Recursive Feature Elimination (RFE) with a number of selected features
rfe = RFE(model, n_features_to_select = 10)
rfe.fit(X_train_val, y_train_val)

# Get the selected features and safe in a list
rfe_features2 = list(X_train_val.columns[rfe.support_])
print(rfe_features2)

In [ ]:
# Fourth method using forward feature selection to determine most relevant features defining n_features_to_select

# Initialize model
model = RandomForestRegressor(n_jobs=-1, random_state=42)

# Forward Feature Selection
sfs_forward = SequentialFeatureSelector(model, n_features_to_select=10, direction='forward')
sfs_forward.fit(X_train_val, y_train_val)

# Selected features from forward selection and print
forward_features2 = list(X_train_val.columns[sfs_forward.get_support()])
print(forward_features2)

## 5. Predictive Modeling

The predictive modeling section was divided into three subchapters. The purpose of this thesis is to apply a multitude of methods to answer the research question successfully. This project used predictive modeling to forecast dengue incidence based on environmental variables using supervised learning for regression. The complexity of this multivariate time-series analysis lies in processing data from multiple Southeast Asia locations, each with different time ranges.


Scale-variant and invariant models were implemented for the assessment. 

**Scale-invariant models:**

Decision Trees (DT) and Random Forests (RF)
- Capable of capturing non-linear relationships and interactions between features.
- Prone to overfitting, especially with deep trees; Random Forests are more robust than single-decision trees.
- Generally invariant to outliers.

XGBoost
- Machine learning algorithms under the Gradient Boosting framework. 
- Less prone to overfitting due to parallel tree boosting than single Decision Trees.
- Generally robust, often performing well in regression tasks.

AdaBoost
- From class sklearn.ensemble
- Does not require feature scaling (Here, I tested both alternatives, and the best results were achieved with the unscaled dataset)


**Scale-variant models:**

SVM (Support Vector Machine)
- Effective for capturing non-linear relationships and handling high-dimensional data.
- Applicable to regression tasks using non-linear kernels.
- Sensitive to outliers.

K-Nearest Neighbors (KNN)
- Sensitive to outliers due to its focus on the distance between data points.

### 5.1 Hyperparameter tuning

The fine-tuning process to improve modeling success.

#### a. Scale-invariant models

*DecisionTreeRegressor*:
max_depth: Maximum depth of the tree
min_samples_split: Minimum number of samples required to split an internal node
min_samples_leaf: Minimum number of samples required to be at a leaf node

*RandomForestRegressor (in addition to DecisionTreeRegressor parameters)*:
n_estimators: Number of trees in the forest
max_features: Number of features to consider when looking for the best split

*XGBRegressor*:
learning_rate: Step size shrinkage used in updates to prevent overfitting
max_depth: Maximum depth of a tree
n_estimators: Number of boosting rounds

*AdaBoostRegressor (similar to XGBRegressor, but with different boosting parameters)*:
learning_rate: Shrinks the contribution of each weak learner
n_estimators: Number of weak learners to train

In [ ]:
## Performing GRidSearchCV to determine optimal parameters
## Here: for AdaBoostRegressor
#
## Define the grid of hyperparameters to search
#param_grid = {
#    'n_estimators': [50, 100, 200],  # Number of boosting stages
#    'learning_rate': [0.01, 0.02],  # Learning rate shrinks the contribution of each regressor
#    'loss': ['linear', 'square', 'exponential']
#}
#
## Set up the GridSearchCV object
#grid_search = GridSearchCV(AdaBoostRegressor(), param_grid, cv=5, n_jobs=-1, verbose=1)
#
## Fit the grid search to the training+validation data
#grid_search.fit(X_train_val, y_train_val)
#
## Print the best parameters and the best score
#print(f"Best parameters found: {grid_search.best_params_}")
#print(f"Best cross-validation score: {grid_search.best_score_:.3f}")
#
## Evaluate the best model on the test set
#best_model = grid_search.best_estimator_
#test_score = best_model.score(X_test, y_test)
#print(f"Test set score with best parameters: {test_score:.3f}")

#### b. Scale-variant models

*KNeighborsRegressor*:
n_neighbors: Number of neighbors to consider.
weights: Weight function used in prediction ('uniform' or 'distance').

*SVR*:
kernel: Specifies the kernel type to be used ('linear', 'poly', 'rbf', 'sigmoid', etc.).
C: Regularization parameter.
gamma: Kernel coefficient for 'rbf', 'poly', and 'sigmoid'.

### 5.2 Cross-Validation and Model Evaluation on Training Set
In the following, the training phase is initialized to evaluate selected models. A time-based cross-validation is applied to handle multiple independent time series for the final independent approach.  The utilized Model Evaluation Metrics are explained as followed: 
- https://www.geeksforgeeks.org/machine-learning-model-evaluation/

#### Holistic approach with a dependent assumption of the variables

Inspiration of dependent/independent approach: 
- https://cienciadedatos.net/documentos/py44-multi-series-forecasting-skforecast.html#:~:text=In%20independent%20multi%2Dseries%20forecasting,as%20predictors%20of%20other%20series
- https://skforecast.org/0.8.0/user_guides/independent-multi-time-series-forecasting

In [ ]:
# Holistic approach 1
# Performing predictive modeling with a generalized model for all locations
# Train Validation Split used with Location Code as differentiator established through LabelEncoder()
# Plot actual vs. predicted values of all models filtered by each location

def evaluate_models(X_train, X_val, y_train, y_val, models):
    # Initialize dictionary to store evaluation metrics
    evaluation_metrics = {}
    
    # Initialize dictionary to store predictions for each model
    predictions = {}
    
    # Iterate through each model
    for name, model in models.items():
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predict on the val set
        y_pred = model.predict(X_val)
        
        # Store predictions in the dictionary
        predictions[name] = y_pred
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mape = mean_absolute_percentage_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        n = X_val.shape[0]
        p = X_val.shape[1]
        adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
        
        # Update evaluation metrics for the model
        evaluation_metrics[name] = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'Adjusted R-squared': adjusted_r2,
        }
        
        print(f"Model: {name}")
        print("Mean Absolute Error (MAE):", mae)
        print("Root Mean Squared Error (RMSE):", rmse)
    
    # Plot actual vs predicted values for each location
    for location in X_val['Location Code'].unique():
        plt.figure(figsize=(15, 10))
        
        # Plot actual values
        y_val_location = y_val[X_val['Location Code'] == location]
        plt.plot(y_val_location.index, y_val_location, label='Actual (y_val)', linewidth=3)
        
        # Plot predicted values from each model
        for name, y_pred in predictions.items():
            y_pred_location = y_pred[X_val['Location Code'] == location]
            plt.plot(y_val_location.index, y_pred_location, label=f'Predicted ({name})')
        
        plt.title(f'Comparison of Actual and Predicted Values for {location}')
        plt.xlabel('Date')
        plt.ylabel('Target Variable')
        plt.legend()
        plt.show()
    
    return evaluation_metrics

#### Individual approach with a independent assumption of the variables


In [ ]:
# The function returns both location-specific metrics and aggregated metrics
# Plot actual vs predicted values for all models in the current location

def evaluate_models(X_train_val, y_train_val, models):
    # Initialize dictionary to store evaluation metrics for each location and overall
    location_metrics = {}
    overall_metrics = {name: {'MAE': [], 'RMSE': []} for name in models.keys()}

    for location in X_train_val['Location Code'].unique():
        location_mask = X_train_val['Location Code'] == location
        location_data = X_train_val[location_mask].drop(columns=['Location Code'])
        location_target = y_train_val[location_mask]

        tscv = TimeSeriesSplit(n_splits=5)
        
        metrics = {name: {'MAE': [], 'RMSE': []} for name in models.keys()}
        predictions_for_plotting = {name: [] for name in models.keys()} 

        for name, model in models.items():
            for train_index, val_index in tscv.split(location_data):
                X_train, X_val = location_data.iloc[train_index], location_data.iloc[val_index]
                y_train, y_val = location_target.iloc[train_index], location_target.iloc[val_index]

                model.fit(X_train, y_train)
                y_pred = model.predict(X_val)
                y_pred = pd.Series(y_pred, index=y_val.index)
                predictions_for_plotting[name].append(y_pred)

                # Calculate and store location metrics for each fold
                mae = mean_absolute_error(y_val, y_pred)
                rmse = np.sqrt(mean_squared_error(y_val, y_pred))
                #mape = mean_absolute_percentage_error(y_val, y_pred)
                #r2 = r2_score(y_val, y_pred)
                #n = X_val.shape[0]
                #p = X_val.shape[1]
                #adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

                metrics[name]['MAE'].append(mae)
                metrics[name]['RMSE'].append(rmse)
                #metrics[name]['MAPE'].append(mape)
                #metrics[name]['Adjusted R-squared'].append(adjusted_r2)

            # Average metrics after all splits for this location
            for metric in metrics[name]:
                metrics[name][metric] = round(np.mean(metrics[name][metric]), 2)
                overall_metrics[name][metric].append(metrics[name][metric])  # Collect metrics across locations

            # Print the averaged metrics
            print(f"Averaged metrics for model {name} at location {location}: {metrics[name]}")

        # Plotting after cross-validation for each location and model
        plt.figure(figsize=(15, 10))
        plt.plot(location_target.index, location_target, label='Actual', linewidth=3)
        for name in models.keys():
            all_preds = pd.concat(predictions_for_plotting[name])
            plt.plot(all_preds.index, all_preds, label=f'Predicted by {name}')
        plt.title(f'Actual vs Predicted Values for Location {location}')
        plt.xlabel('Date')
        plt.ylabel('Incidence Rate')
        plt.legend()
        plt.show()
            
        location_metrics[location] = metrics

    # Calculate and print overall average metrics for each model across all locations
    for name, metrics in overall_metrics.items():
        for metric in metrics:
            overall_metrics[name][metric] = round(np.mean(metrics[metric]), 2) if metrics[metric] else np.nan
        print(f"Overall averaged metrics for model {name}: {overall_metrics[name]}")

    return location_metrics, overall_metrics

In [ ]:
def visualize_splits(X_train_val, y_train_val, location):
    location_mask = X_train_val['Location Code'] == location
    location_data = X_train_val[location_mask].drop(columns=['Location Code'])
    location_target = y_train_val[location_mask]

    tscv = TimeSeriesSplit(n_splits=5)
    fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(15, 20))
    fig.suptitle(f'Time Series Cross-Validation for Location {location}', fontsize=18)

    # Determining the fixed x-axis range based on the whole dataset
    x_min, x_max = location_data.index.min(), location_data.index.max()
    y_min, y_max = location_target.min(), location_target.max()
    
    for i, (train_index, val_index) in enumerate(tscv.split(location_data)):
        X_train, X_val = location_data.iloc[train_index], location_data.iloc[val_index]
        y_train, y_val = location_target.iloc[train_index], location_target.iloc[val_index]

        axes[i].plot(X_train.index, y_train, 'blue', label='Training data')
        axes[i].plot(X_val.index, y_val, 'red', label='Validation data')
        axes[i].set_title(f'Split {i+1}')
        axes[i].set_xlim([x_min, x_max])
        axes[i].set_ylim([y_min, y_max])
        axes[i].set_ylabel('Incidence Rate')
        axes[i].set_xlabel('Date')
        axes[i].legend()
        
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(f"{location}_TimeSeriesSplit.png", dpi=200, bbox_inches='tight')
    plt.show()

visualize_splits(X_train_val, y_train_val, 11)

#### a. Scale-invariant models

In [ ]:
# Defining the best parameters for the modeling section
best_params = {
    'Decision Tree':{},
    'Random Forest':{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10},
    'XGBoost':{'colsample_bytree': 0.7, 'learning_rate': 0.014, 'max_depth': 9, 'min_child_weight': 2, 'n_estimators': 141, 'reg_lambda': 3, 'subsample': 0.956},
    'AdaBoost':{'learning_rate': 0.01, 'loss': 'linear'}
}

# Define chosen models for unscaled datasets
scale_invariant_models = {
    'Decision Tree': DecisionTreeRegressor(random_state=42, **best_params['Decision Tree']), # captures non-linear relationships, prone to overfitting
    'Random Forest': RandomForestRegressor(random_state=42, **best_params['Random Forest']), #bagging ensemble method
    'XGBoost': XGBRegressor(random_state=42, **best_params['XGBoost']), # boosting ensemble method, improves weak learners, handles non-linear relationships, less prone to overfitting
    'AdaBoost': AdaBoostRegressor(random_state=42, **best_params['AdaBoost']), # boosting ensemble method, improve weak learners, handle non-linear relationships, less prone to overfitting
   }

In [ ]:
# Performing modeling for tree-based, scale-invariant models on all variables

evaluation_metrics_original = evaluate_models(X_train_val, y_train_val, scale_invariant_models)

In [ ]:
# Performing modeling for tree-based, scale-invariant models using rf feature selection

evaluation_metrics_rf = evaluate_models(X_train_val[rf_features], y_train_val, scale_invariant_models)

In [ ]:
# Performing modeling for tree-based, scale-invariant models using xgb feature selection

evaluation_metrics_xgb = evaluate_models(X_train_val[xgb_features], y_train_val, scale_invariant_models)

In [ ]:
# Performing modeling for tree-based, scale-invariant models using rfe feature selection

evaluation_metrics_rfe2 = evaluate_models(X_train_val[rfe_features2], y_train_val, scale_invariant_models)

In [ ]:
# Performing modeling for tree-based, scale-invariant models using forward feature selection

evaluation_metrics_forward2 = evaluate_models(X_train_val[forward_features2], y_train_val, scale_invariant_models)

#### b. Scale-variant models

In [ ]:
# Defining the best parameters for the modeling section
best_params = {
    #'Logistic Regression':,
    'KNN':{'metric': 'euclidean', 'n_neighbors': 10, 'weights': 'distance'},
    'Support Vector Regression':{'C': 100, 'kernel': 'rbf'}
}

# Define a dictionary to hold models
scale_variant_models = {
    #'Logistic Regression': LogisticRegression(**best_params['Logistic Regression']),
    'KNN': KNeighborsRegressor(**best_params['KNN']), # makes predictions based on the similarity of data points
    'Support Vector Regression': SVR(**best_params['Support Vector Regression']) #captures non-linear relationships and handling high-dimensional data
}

#### MinMaxScaler

Perform modeling for scale-variant models, using a dataset normalized with the MinMaxScaler.


Source: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [ ]:
# Performing modeling for scale-variant models using minmax scaler on the original dataset

evaluation_metrics_minmax_original = evaluate_models(X_train_val_scaled_minmax, y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using minmax scaler on the rf dataset

evaluation_metrics_minmax_rf = evaluate_models(X_train_val_scaled_minmax[rf_features], y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using minmax scaler on the xgb dataset

evaluation_metrics_minmax_xgb = evaluate_models(X_train_val_scaled_minmax[xgb_features], y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using minmax scaler on the rfe dataset with 10 features

evaluation_metrics_minmax_rfe2 = evaluate_models(X_train_val_scaled_minmax[rfe_features2], y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using minmax scaler on the forward selection dataset with 10 features
evaluation_metrics_minmax_forward2 = evaluate_models(X_train_val_scaled_minmax[forward_features2], y_train_val, scale_variant_models)

#### RobustScaler
Perform modeling for scale-variant models, using a dataset normalized with the RobustScaler.

Source: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html

In [ ]:
# Performing modeling for scale-variant models using robust scaler on the original dataset

evaluation_metrics_robust_original = evaluate_models(X_train_val_scaled_robust, y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using robust scaler on the rf dataset

evaluation_metrics_robust_rf = evaluate_models(X_train_val_scaled_robust[rf_features], y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using robust scaler on the xgb dataset

evaluation_metrics_robust_xgb = evaluate_models(X_train_val_scaled_robust[xgb_features], y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using robust scaler on the rfe dataset with 10 features

evaluation_metrics_robust_rfe2 = evaluate_models(X_train_val_scaled_robust[rfe_features2], y_train_val, scale_variant_models)

In [ ]:
# Performing modeling for scale-variant models using robust scaler on the forward selection dataset with 5 features

evaluation_metrics_robust_forward2 = evaluate_models(X_train_val_scaled_robust[forward_features2], y_train_val, scale_variant_models)

### 5.3 Final Training and Prediction on Test Set

Predict unseen data to evaluate model effectiveness in forecasting dengue incidence rates.

In [ ]:
# Function for the test set

def train_and_evaluate_on_test_set(X_train_val, y_train_val, X_test, y_test, models):
    
    test_metrics = {}
    test_predictions = {}
    
    overall_metrics = {name: {'MAE': [], 'RMSE': []} for name in models.keys()}

    for location in X_train_val['Location Code'].unique():
        
        location_mask_train = X_train_val['Location Code'] == location
        location_data_train = X_train_val[location_mask_train].drop(columns=['Location Code'])
        location_target_train = y_train_val[location_mask_train]

        location_mask_test = X_test['Location Code'] == location
        location_data_test = X_test[location_mask_test].drop(columns=['Location Code'])
        location_target_test = y_test[location_mask_test]

        metrics = {name: {'MAE': None, 'RMSE': None} for name in models.keys()}
        predictions_for_plotting = {name: [] for name in models.keys()}

        for name, model in models.items():
            model.fit(location_data_train, location_target_train)
            y_pred_test = model.predict(location_data_test)
            y_pred_test = pd.Series(y_pred_test, index=location_target_test.index)
            predictions_for_plotting[name].append(y_pred_test)
            test_predictions.setdefault(name, {})[location] = y_pred_test

            mae = round(mean_absolute_error(location_target_test, y_pred_test), 2)
            rmse = round(np.sqrt(mean_squared_error(location_target_test, y_pred_test)), 2)

            metrics[name]['MAE'] = mae
            metrics[name]['RMSE'] = rmse

            overall_metrics[name]['MAE'].append(mae)
            overall_metrics[name]['RMSE'].append(rmse)

            print(f"Test metrics for model {name} at location {location}: MAE={mae}, RMSE={rmse}")

        plt.figure(figsize=(15, 10))
        plt.plot(location_target_test.index, location_target_test, label='Actual', linewidth=3)
        for name in models.keys():
            plt.plot(predictions_for_plotting[name][0].index, predictions_for_plotting[name][0], label=f'Predicted by {name}')
        plt.title(f'Actual vs Predicted Values for Location {location}')
        plt.xlabel('Date')
        plt.ylabel('Incidence Rate')
        plt.legend()
        plt.show()

        test_metrics[location] = metrics

    # Calculate and print overall average metrics for each model across all locations
    for name, metrics in overall_metrics.items():
        for metric in metrics:
            overall_metrics[name][metric] = round(np.mean(metrics[metric]) if metrics[metric] else np.nan, 2)
        print(f"Overall averaged metrics for model {name}: {overall_metrics[name]}")

    return test_metrics, test_predictions, overall_metrics

In [ ]:
# Defining the best parameters for the modeling section
best_params = {
    'Support Vector Regression':{'C': 100, 'kernel': 'rbf'},
    'AdaBoost':{'learning_rate': 0.01, 'loss': 'linear'}
}

# Define a dictionary to hold models
scale_variant_models = {
    'Support Vector Regression': SVR(**best_params['Support Vector Regression']) #captures non-linear relationships and handling high-dimensional data
}

# Define chosen models for unscaled datasets
scale_invariant_models = {
    'AdaBoost': AdaBoostRegressor(random_state=42, **best_params['AdaBoost']), # boosting ensemble method, improve weak learners, handle non-linear relationships, less prone to overfitting
   }

#### a. Scale-invariant models

In [ ]:
test_metrics, test_predictions, overall_metrics = train_and_evaluate_on_test_set(X_train_val[rf_features], y_train_val, X_test[rf_features], y_test, scale_invariant_models)

#### b. Scale-variant models

In [ ]:
test_metrics, test_predictions, overall_metrics = train_and_evaluate_on_test_set(X_train_val_scaled_robust , y_train_val, X_test_scaled_robust, y_test, scale_variant_models)